In [1]:
import os

In [2]:
from pathlib import Path

In [16]:
%reload_ext autoreload
%autoreload 2


from tabpfn_time_series.experimental.finetuning.data.data_module import (
    TimeSeriesDataModule,
)
from tabpfn import TabPFNRegressor

model = TabPFNRegressor(n_estimators=3)

data_module = TimeSeriesDataModule(
    dataset_name="jena_weather/H",
    dataset_storage_path=Path(os.getenv("DATASET_STORAGE_PATH")),
    model=model,
    num_workers=0,
    batch_size=1,
)
data_module.setup("fit")

In [17]:
os.environ["TABPFN_ALLOW_CPU_LARGE_DATASET"] = "1"

In [18]:
train_dataloader = data_module.train_dataloader()

for i, sample in enumerate(train_dataloader):
    print(sample)
    break

/Users/shibinhoo/Documents/freiburg-cs/tabpfn-time-series/tabpfn_time_series/experimental/finetuning/TabPFN/src/tabpfn/model/loading.py:672: UserWarning: Fields in config not in Config class: {'adaptive_max_seq_len_to_max_full_table_size', 'batch_size', 'seq_len', 'num_global_att_tokens', 'multi_query_factor', 'task_type', 'differentiable_hps_as_style', 'semisupervised_enabled', 'aggregate_k_gradients', 'timing', 'progress_bar'}
  config = ModelConfig.from_dict(ModelConfig.upgrade_config(checkpoint["config"]))
/Users/shibinhoo/Documents/freiburg-cs/tabpfn-time-series/.venv/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/Users/shibinhoo/Documents/freiburg-cs/tabpfn-time-series/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: F

{'X_train_preprocessed': [[tensor([[ 1.0000,  0.4678,  0.0000,  ...,  0.0000,  1.0000,  0.0000],
        [ 0.8961,  0.7657,  0.0000,  ...,  0.0000,  1.0000,  0.2698],
        [ 0.7921,  0.0196,  0.0000,  ...,  0.0000,  1.0000,  0.5196],
        ...,
        [ 0.1461,  0.0115,  0.4965,  ...,  0.5264,  0.8230, -0.3984],
        [ 0.1875,  0.9875,  0.4965,  ...,  0.5264,  0.8230, -0.6311],
        [ 0.2718,  0.9578,  0.4965,  ...,  0.5264,  0.8230, -0.8170]]), tensor([[ 8.6603e-01,  5.5895e+00,  7.7297e-01,  ...,  6.2118e+00,
          0.0000e+00,  0.0000e+00],
        [ 8.6603e-01,  5.5765e+00,  3.6632e-01,  ...,  6.2091e+00,
          0.0000e+00,  0.0000e+00],
        [ 8.6603e-01,  5.5621e+00,  1.7124e-02,  ...,  6.2053e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [ 1.2246e-16,  4.9958e+00,  1.9784e-01,  ..., -3.2332e+00,
          2.3455e-01,  2.3736e-01],
        [ 1.2246e-16,  4.9625e+00,  8.1555e-01,  ..., -3.2433e+00,
          2.3455e-01,  2.3736e-01],
        [ 

In [20]:
for k in sample.keys():
    print(k)
    print(len(sample[k]))

X_train_preprocessed
1
y_train_standardized
1
X_test_preprocessed
1
y_test_standardized
1
cat_ix
1
configs
1
bardist
1
normalized_bardist
1
x_test_raw
1
y_test_raw
1


In [26]:
original_params = {
    name: param.clone().detach() for name, param in model.model_.named_parameters()
}

In [28]:
original_params

{'encoder.5.layer.weight': tensor([[-3.6619e-01,  3.3757e-01, -1.4622e-01,  8.8530e-02],
         [ 1.5249e-01, -7.0819e-03, -2.2418e-01, -2.5865e-01],
         [-6.0513e-01, -4.1899e-01, -2.4611e-01, -2.8159e-01],
         [-4.4921e-01, -4.1392e-01,  1.1176e-01, -3.7108e-01],
         [-6.4187e-01, -4.0271e-02,  2.9327e-01, -1.0378e-01],
         [-2.0177e-02,  2.8500e-02,  3.5429e-01, -2.5923e-01],
         [-5.0081e-02,  4.9181e-02, -2.4641e-01, -8.7560e-02],
         [ 7.2205e-02, -2.8625e-01, -3.4707e-02,  2.3148e-01],
         [-9.3626e-02,  4.6098e-01, -1.2391e-02,  3.6334e-01],
         [-2.1322e-01,  3.7015e-01, -1.0827e-01, -2.1714e-01],
         [ 2.0581e-01, -1.5710e-01, -4.8419e-01, -4.0737e-01],
         [ 4.2919e-02, -1.5060e-02,  4.7514e-02,  1.3674e-01],
         [-1.0247e-02, -7.2823e-03, -5.0406e-01, -6.7887e-02],
         [ 6.1419e-02, -1.1230e-01,  4.9028e-02, -3.3488e-01],
         [ 4.8172e-01,  9.4570e-01, -9.3319e-02,  4.2539e-01],
         [-3.0153e-01,  3.876

In [23]:
from tabpfn_time_series.experimental.finetuning.data.raw_time_series_dataset import (
    RawTimeSeriesDataset,
)


from pathlib import Path
from tabpfn_time_series.experimental.evaluation.data import GiftEvalDataset


jena_weather_dataset = GiftEvalDataset(
    # name="ett1/15T",
    # name="bizitobs_service",
    name="jena_weather/H",
    storage_path=Path(os.getenv("DATASET_STORAGE_PATH")),
    to_univariate=True,
)

raw_ts_dataset = RawTimeSeriesDataset(
    datasets=[jena_weather_dataset.training_dataset],
    probabilities=[1.0],
    past_length=10_000,
    future_length=1_024,
    min_future=jena_weather_dataset.prediction_length,
)

In [ ]:
sample = next(iter(jena_weather_dataset.training_dataset))
print(sample)